# This notebook shows the training pipeline for Pose Bowl: Detection Track competition to train yolov8s model.Following are few points to consider:
- replace the DATA_DIR folder with your own data directory where data is stored
- replace the ROOT_DIR folder with your current working directory. In case both are same then ROOT_DIR = DATA_DIR
- during training each epoch checkpoint will be saved to ROOT_DIR + 'fold0/train' and epoch28 checkpoint will be used for inference

In [1]:
from ultralytics import YOLO
import pandas as pd
import os, cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob, shutil
from joblib import Parallel, delayed
import torch
import yaml, gc
from sklearn.model_selection import StratifiedKFold, GroupKFold
from utils import coco2yolo, coco2voc, voc2yolo
from utils import draw_bboxes, load_image
from utils import clip_bbox, str2annot, annot2str

/home/ashish/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create Labels
We need to export our labels to **YOLO** format, with one `*.txt` file per image (if no objects in image, no `*.txt` file is required). The *.txt file specifications are:

* One row per object
* Each row is class `[x_center, y_center, width, height]` format.
* Box coordinates must be in **normalized** `xywh` format (from `0 - 1`). If your boxes are in pixels, divide `x_center` and `width` by `image width`, and `y_center` and `height` by `image height`.
* Class numbers are **zero-indexed** (start from `0`).

> Competition bbox format is **pascal voc** hence `[x_min, y_min, width, height]`. So, we need to convert form **pascal voc** to **YOLO** format.

In [2]:
# code 
def create_label_file(df):
    cnt = 0
    all_bboxes = []
    bboxes_info = []
    for row_idx in tqdm(range(df.shape[0])):
        row = df.iloc[row_idx]
        image_height = row.img_height
        image_width  = row.img_width
        bboxes_voc  = np.array([[row.xmin, row.ymin, row.xmax, row.ymax]]).astype(np.float32).copy()
        num_bbox     = len(bboxes_voc)
        names        = ['satellite']*num_bbox
        labels       = np.array([0]*num_bbox)[..., None].astype(str)
        ## Create Annotation(YOLO)
        with open(row.label_path, 'w') as f:
            if num_bbox<1:
                annot = ''
                f.write(annot)
                cnt+=1
                continue
            bboxes_voc  = clip_bbox(bboxes_voc, image_height, image_width)
            bboxes_yolo = voc2yolo(bboxes_voc, image_height, image_width).astype(str)
            all_bboxes.extend(bboxes_yolo.astype(float))
            bboxes_info.extend([[row.image_id]]*len(bboxes_yolo))
            annots = np.concatenate([labels, bboxes_yolo], axis=1)
            string = annot2str(annots)
            f.write(string)
    print('Missing:',cnt)

## # Create Folds
> The following code split spacecraft_id and background_id into 25 folds each containing unique ids that do not intersect with other folds.I have used only fold 0 for validation and it perfectly coorelates with the public leaderboard.

In [3]:
def create_validation_set(df):
    kf = GroupKFold(n_splits = 5)
    df['kfold'] = -1
    for fold, (train_idx, val_idx) in enumerate(kf.split(df['image_id'], y = df['spacecraft_id'], groups = df['background_id'])):   
        df.loc[val_idx, 'kfold'] = fold

    j = -1
    for i, ids in enumerate(np.array_split(df['spacecraft_id'].unique(), indices_or_sections = 5)):
        for k in range(5):
            j += 1
            df.loc[(df['spacecraft_id'].isin(ids)) & (df['background_id'].isin(df[df['kfold'] == k].background_id.unique())), 'fold'] = j
    return df

#  Write Images
* We need to copy the Images to Current Directory as training data are split into multiple files.
* We can make this process faster using **Joblib** which uses **Parallel** computing.

In [4]:
def make_copy(row):
    shutil.copyfile(row.old_image_path, row.image_path)
    return

In [5]:
#create folder to save images and labels in current directory

def create_folder(IMAGE_DIR, LABEL_DIR):
    if os.path.exists(IMAGE_DIR):
        !rm -rf {IMAGE_DIR}
        !rm -rf {LABEL_DIR}
    !mkdir -p {IMAGE_DIR}
    !mkdir -p {LABEL_DIR}

#  Configuration
The dataset config file requires
1. The dataset root directory path and relative paths to `train / val / test` image directories (or *.txt files with image paths)
2. The number of classes `nc` and 
3. A list of class `names`:`['sat']`

In [6]:
def get_yaml_file(train_df, valid_df, cwd,fold):
    with open(os.path.join( cwd , 'train.txt'), 'w') as f:
        for path in train_df.image_path.tolist():
            f.write(path+'\n')

    with open(os.path.join(cwd , 'val.txt'), 'w') as f:
        for path in valid_df.image_path.tolist():
            f.write(path+'\n')

    data = dict(
        train =  os.path.join( cwd , 'train.txt'),
        val   =  os.path.join( cwd , 'val.txt' ),
        nc    = 1,
        names = ['sat'],
        )

    with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)
        
    f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
    print('\nyaml:')
    print(f.read())

In [13]:
def save_valid_df(valid_df, fold):
#     valid_df.reset_index(drop = True).to_csv('code_execution/data/submission_format.csv', index = False)
    COLUMNS = ['image_id', 'xmin', 'ymin', 'xmax', 'ymax']
    valid_df[COLUMNS].to_csv( ROOT_DIR + f'fold{fold}/fold0_true_df.csv', index = False)

In [8]:
# import yaml
# with open('hyp.yml') as file:
#     hyp = yaml.safe_load(file)

In [9]:
DATA_DIR = '/home/ashish/satellite/'  # 
ROOT_DIR = '/home/ashish/solution/'

# training loop

In [10]:
if __name__ == '__main__':
    fold = 0
    df = pd.read_csv(DATA_DIR + 'train_labels.csv')
    metadata = pd.read_csv(DATA_DIR + 'train_metadata.csv')
    df = pd.merge(df, metadata, on = 'image_id').reset_index(drop = True)
    df['id'] = df.image_id.str[0]
    df['old_image_path']  = DATA_DIR +df.image_id.str[0] + '/images/' + df.image_id + '.png'
    df['img_height'] = 1024
    df['img_width'] = 1280

    df = create_validation_set(df) # create folds

    IMAGE_DIR = ROOT_DIR + f'fold{fold}/images' # directory to save images
    LABEL_DIR = ROOT_DIR + f'fold{fold}/labels' # directory to save labels
    
    df['image_path']  = f'{IMAGE_DIR}/' +  df.image_id + '.png' # image and label paths to read images and labels by yolov8
    df['label_path']  = f'{LABEL_DIR}/' +  df.image_id + '.txt'
    
    valid_df = df[df['fold'] == fold]   #yolo needs both val and train txt files for training
#     train_df = df[(~df['spacecraft_id'].isin(valid_df.spacecraft_id.unique())) & 
#                   (~df['background_id'].isin(valid_df.background_id.unique()))].reset_index(drop = True)   
    train_df = df.copy()
    create_folder(IMAGE_DIR, LABEL_DIR)
#     save_valid_df(valid_df, fold)  used for validating the model on oof
    
    # copying training images from multiple directory into single directory
    image_paths = df.old_image_path.tolist()
    _ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row) for _, row in tqdm(df.iterrows(), total=len(df)))   
    create_label_file(df)
    print('fold : ', fold, 'train_images : ', len(train_df),'val_images : ', len(valid_df))
    cwd = ROOT_DIR + f'fold{fold}'
    get_yaml_file(train_df, valid_df, cwd,fold) #get configuration file
    
    model = YOLO('yolov8s.pt', task = 'detect')  #define and load pretrained yolov8s model
    results = model.train(data=os.path.join(cwd , 'gbr.yaml'),cfg = 'hyp.yml', project = ROOT_DIR + f'fold{fold}') #start training

100%|██████████| 25801/25801 [00:04<00:00, 5308.88it/s]


Missing: 0
fold :  0 train_images :  25801 val_images :  1089

yaml:
names:
- sat
nc: 1
train: /home/ashish/drivendata/solution/fold0/train.txt
val: /home/ashish/drivendata/solution/fold0/val.txt

New https://pypi.org/project/ultralytics/8.2.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.1.2 CUDA:0 (NVIDIA RTX A6000, 48655MiB)
engine/trainer: task=detect, mode=train, model=None, data=/home/ashish/drivendata/solution/fold0/gbr.yaml, epochs=30, time=None, patience=100, batch=32, imgsz=1280, save=True, save_period=1, cache=False, device=None, workers=8, project=/home/ashish/drivendata/solution/fold0, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=5, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, sav

train: Scanning /home/ashish/drivendata/solution/fold0/labels... 25801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25801/25801 [01:52<00:00, 228.87it/s]


train: New cache created: /home/ashish/drivendata/solution/fold0/labels.cache
albumentations: Blur(p=0.001, blur_limit=(3, 7)), MedianBlur(p=0.001, blur_limit=(3, 7)), ToGray(p=0.001), CLAHE(p=0.001, clip_limit=(1, 4.0), tile_grid_size=(8, 8)), RandomRotate90(p=0.5)


val: Scanning /home/ashish/drivendata/solution/fold0/labels... 1089 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1089/1089 [00:05<00:00, 191.37it/s]


val: New cache created: /home/ashish/drivendata/solution/fold0/labels.cache
Plotting labels to /home/ashish/drivendata/solution/fold0/train/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to /home/ashish/drivendata/solution/fold0/train
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/807 [00:00<?, ?it/s]

16


       1/30      28.5G      2.483       26.3      7.752         39       1280:   0%|          | 3/807 [00:04<18:28,  1.38s/it]
Exception in thread Thread-81 (_pin_memory_loop):
Traceback (most recent call last):
  File "/home/ashish/anaconda3/envs/pytorch/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/ashish/anaconda3/envs/pytorch/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ashish/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 54, in _pin_memory_loop
    do_one_step()
  File "/home/ashish/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 31, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/ashish/anaconda3/envs/pytorch/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/home/ashish/anaconda3/envs/p

KeyboardInterrupt: 

In [14]:
# save_valid_df(valid_df, fold)